# Direct Preference Optimization

## DPO：动机（为什么要DPO？）

- 在 RLHF 的典型流水线中，需要先训练一个 **reward model** (RM)，然后基于 RM 用 RL（如 PPO）训练策略（policy）。这套流程存在的痛点：
  1. **复杂与不稳定**：RM 训练 + RL（采样、PPO 超参等）导致流程复杂且敏感超参。
  2. **计算/实现成本**：训练 RM、在线采样、PPO 的多轮 actor-critic 更新都很耗资源。
  3. **RM 的偏差会被 policy exploit**：policy 可能学习去 exploit RM 的弱点，产生不自然文本，因此通常还需强 KL 惩罚与工整的超参调节。

- **DPO** (Direct Preference Optimization) 的核心思想是：**重参数化 reward model**，使得在一定假设下可以直接从偏好对 (x, y_w, y_l) 得到一个可用于训练 LM 的监督式损失，
而无需单独训练 reward model + 无需基于采样的 RL 优化。

### (1). 背景与关键假设（出发点）

- 有参考（behavior）策略 / SFT：$\pi_{\text{ref}}(y \mid x)$。
- 我们考虑一个 RL 目标（带 KL 惩罚的期望 reward）：
  $$
  J(\pi) = \mathbb{E}_x \left[ \mathbb{E}_{y \sim \pi(\cdot \mid x)} \left[ r(x, y) \right] - \beta \, \text{KL}(\pi(\cdot \mid x) \parallel \pi_{\text{ref}}(\cdot \mid x)) \right],
  $$
  其中 $\beta > 0$ 为权衡系数（类似热度 / 惩罚强度）。

- 假设人类偏好产生机制（常用建模）：对于同一 $x$，两条候选回答 $y_w$（winner）与 $y_l$（loser），
  $$
  P(y_w \succ y_l \mid x) = \sigma(r(x, y_w) - r(x, y_l)),
  $$
  其中 $\sigma(z) = \frac{1}{1 + e^{-z}}$。

### (2). 最优策略的解析解（求 $\pi^*$）

最大化 $J(\pi)$（对每个 $x$ 独立）可视为变分问题。对每个 $x$ 求解：

$$
\pi^*(\cdot \mid x) = \arg \max_{\pi(\cdot \mid x)} \mathbb{E}_{y \sim \pi} [r(x, y)] - \beta \text{KL}(\pi \parallel \pi_{\text{ref}})
$$

用拉格朗日或直接变分可得 Gibbs 形式的解析解（标准结果）：

$$
\pi^*(y \mid x) \propto \pi_{\text{ref}}(y \mid x) \exp \left( \frac{r(x, y)}{\beta} \right).
$$

因此对数概率满足（去掉与 $y$ 无关的归一化常数 $Z(x)$）：

$$
\log \pi^*(y \mid x) = \log \pi_{\text{ref}}(y \mid x) + \frac{r(x, y)}{\beta} - \log Z(x).
$$

将上式变形得出 reward 与 policy 的一一线性关系（up to an additive function of $x$）：

$$
r(x, y) = \beta (\log \pi^*(y \mid x) - \log \pi_{\text{ref}}(y \mid x)) + C(x),
$$

其中 $C(x) = \beta \log Z(x)$ 与 $y$ 无关，会在比较差值时抵消掉。

### (3). 将偏好模型代入并消去 $r$

偏好概率为

$$
P(y_w \succ y_l \mid x) = \sigma(r(x, y_w) - r(x, y_l)).
$$

把上面的 $r$ 表达式代入（注意 $C(x)$ 抵消）：

$$
r(x, y_w) - r(x, y_l) = \beta \Big( \log \pi^*(y_w \mid x) - \log \pi^*(y_l \mid x) - (\log \pi_{\text{ref}}(y_w \mid x) - \log \pi_{\text{ref}}(y_l \mid x)) \Big).
$$

于是偏好概率变为

$$
P(y_w \succ y_l \mid x) = \sigma\Big( \beta \Big[ \Delta_{\pi^*}(x) - \Delta_{\pi_{\text{ref}}}(x) \Big] \Big),
$$

其中定义差分

$$
\Delta_\pi(x) \triangleq \log \pi(y_w \mid x) - \log \pi(y_l \mid x).
$$

### (4). 构造 DPO 的损失（极其关键的一步）

若我们把策略使用参数化 $\pi_\phi$ 表示，并假设偏好标签真实服从上述模型，那么偏好数据集上的对数似然为

$$
\mathcal{L}_{\text{pref}}(\phi) = \sum_{(x, y_w, y_l) \in \mathcal{D}} \log \sigma \Big( \beta \big( \Delta_{\pi_\phi}(x) - \Delta_{\pi_{\text{ref}}}(x) \big) \Big).
$$

DPO 就是直接对策略参数 $\phi$ 最大化 这个偏好对数似然（或最小化其负对数）：

$$
\mathcal{L}_{\text{DPO}}(\phi) = - \sum_{(x, y_w, y_l) \in \mathcal{D}} \log \sigma \Big( \beta \big( \Delta_{\pi_\phi}(x) - \Delta_{\pi_{\text{ref}}}(x) \big) \Big).
$$

也可以写成每个样本的平均损失形式：

$$
\ell_{\text{DPO}}(\phi; x, y_w, y_l) = - \log \sigma \Big( \beta \big( (\log \pi_\phi(y_w \mid x) - \log \pi_\phi(y_l \mid x)) - (\log \pi_{\text{ref}}(y_w \mid x) - \log \pi_{\text{ref}}(y_l \mid x)) \big) \Big).
$$

**注意**：
- $\log \pi_{\text{ref}}$ 是已知（SFT 模型），作为偏置项 subtract 出来，等价于把参考策略的对数概率当作 baseline / offset。
- $\beta$ 控制尺度（越大，sigmoid 输入放大，模型越“自信”）；在文献中常把它视为温度或惩罚系数的反比形式，需调参。

这里我们使用 $\hat{r}_{\theta} (x, y)= \beta \log \frac{\pi_{\theta}(x | y)}{\pi_{ref}(x | y)}$，我们得到：
$$
\mathcal{L}_{\text{DPO}}(\pi_\theta; \pi_{\text{ref}}) = -\mathbb{E}_{(x, y_w, y_l) \sim \mathcal{D}} \left[ \log \sigma  \beta \left (\hat{r}_{\theta} (x, y_w) - \hat{r}_{\theta} (x, y_l) \right) \right].
$$
**这里隐式的表明奖励是由语言模型和参考模型定义的。**


# 伪代码
```python
#输入：偏好对数据 D = {(x, y_w, y_l)}, 参考模型 π_ref, 初始策略 π_φ (通常从 π_ref 初始化), 超参 β, 学习率 η 

for epoch in 1..N:
  for batch B ⊂ D:
    # 计算 log-probs（注意用相同 tokenizer & 限制）
    lp_w = log π_φ(y_w | x)   # shape [batch]
    lp_l = log π_φ(y_l | x)
    lp_ref_w = log π_ref(y_w | x)
    lp_ref_l = log π_ref(y_l | x)

    s = β * ( (lp_w - lp_l) - (lp_ref_w - lp_ref_l) )   # shape [batch]
    loss = -mean( log sigmoid(s) )

    更新 φ: φ ← φ - η ∇_φ loss
```

注意:

- 在实践中，人们倾向于重用公开可用的偏好数据集，而不是生成样本并收集人类偏好。由于偏好数据集是使用 $\pi^{SFT}$ 采样的，因此当 $\pi^{SFT}$ 可用时，初始化 $\pi_{ref} = \pi^{SFT}$
- 其中的 D 可以由参考策略 π_ref 采样(y_1, y_2)进行人工筛选得出

# 附录

## 1.推导（逐步一用拉格朗日乘子法，结论是 Gibbs 形式）

目标是对每个固定的 $x$ 最大化

$$
J(\pi(\cdot \mid x)) = \mathbb{E}_{y \sim \pi(\cdot \mid x)} [r(x, y)] - \beta \, \text{KL}(\pi(\cdot \mid x) \parallel \pi_{\text{ref}}(\cdot \mid x)),
$$

并且约束 $\sum_y \pi(y \mid x) = 1$（离散情形）。把期望和 KL 展开成求和：

$$
J = \sum_y \pi(y \mid x) r(x, y) - \beta \sum_y \pi(y \mid x) \log \frac{\pi(y \mid x)}{\pi_{\text{ref}}(y \mid x)}.
$$

用拉格朗日乘子 $\lambda$ 加上归一化约束，构造 Lagrangian

$$
\mathcal{L} = \sum_y \pi(y \mid x) r(x, y) - \beta \sum_y \pi(y \mid x) \log \frac{\pi(y \mid x)}{\pi_{\text{ref}}(y \mid x)} + \lambda \Big( \sum_y \pi(y \mid x) - 1 \Big).
$$

对每个 $y$ 对 $\pi(y \mid x)$ 求偏导并令其为 0：

$$
\frac{\partial \mathcal{L}}{\partial \pi(y \mid x)} = r(x, y) - \beta \Big( \log \frac{\pi(y \mid x)}{\pi_{\text{ref}}(y \mid x)} + 1 \Big) + \lambda = 0.
$$

整理得

$$
\log \frac{\pi(y \mid x)}{\pi_{\text{ref}}(y \mid x)} = \frac{r(x, y) + \lambda - \beta}{\beta}.
$$

右端关于 $y$ 的依赖只来自 $r(x, y)$，其余项是对所有 $y$ 相同的常数。把常数合并为归一化项（写成 $C(x)$），得到

$$
\log \pi(y \mid x) = \log \pi_{\text{ref}}(y \mid x) + \frac{r(x, y)}{\beta} + C(x).
$$

两边取指数并用归一化常数 $Z(x) = e^{-C(x)}$：

$$
\pi(y \mid x) = \pi_{\text{ref}}(y \mid x) \exp \left( \frac{r(x, y)}{\beta} \right) / Z(x).
$$

因此得到常见的 Gibbs（也叫 Boltzmann / softmax）形式：

$$
\pi^*(y \mid x) \propto \pi_{\text{ref}}(y \mid x) \exp \left( \frac{r(x, y)}{\beta} \right),
$$

其中 $Z(x) = \sum_y \pi_{\text{ref}}(y \mid x) \exp(r(x, y) / \beta)$ 保证概率和为 1。

论文里写法更“信息论化”，思路简洁：原问题其实就是把策略 $\pi$ 投影到一个经过 reward reweight 的分布上。所以结论几乎不用求导，直接从 KL 的最优性就能读出。



## 2.梯度直观（把 DPO 看成“带优势信号的监督”）

对单个偏好对损失：

$$
\ell = -\log \sigma \left( \beta (\Delta_\phi - \Delta_{\text{ref}}) \right), \quad \Delta_\phi = \log \pi_\phi(y_w) - \log \pi_\phi(y_l).
$$

令 $s \triangleq \beta (\Delta_\phi - \Delta_{\text{ref}})$。有

$$
\frac{\partial \ell}{\partial \Delta_\phi} = -\beta (1 - \sigma(s)) = -\beta \sigma(-s).
$$

于是对 $\log \pi_\phi(y_w)$ 和 $\log \pi_\phi(y_l)$ 的导数分别为

$$
\frac{\partial \ell}{\partial \log \pi_\phi(y_w)} = -\beta \sigma(-s),
$$

$$
\frac{\partial \ell}{\partial \log \pi_\phi(y_l)} = +\beta \sigma(-s).
$$

直观含义：

- 若 $s$ 很大（模型已倾向 winner），$\sigma(-s)$ 很小，梯度接近 0（收敛）；
- 若 $s$ 小或负，$\sigma(-s)$ 接近 0.5~1，梯度把 $\log \pi_\phi(y_w)$ 往上推、把 $\log \pi_\phi(y_l)$ 往下拉。

这和 policy-gradient 的形式类似：把“优势”（这里由 $\sigma(-s)$ 形式给出）作为权重来调整 log-prob。DPO 的“优势”是一个来自偏好对并受 $\beta$ 缩放的信号。